In [17]:
from datetime import datetime
import config, input_tables
from steps import buildhdf,mktiles,mkphotometry,fow2cells
from stralog import getLogger
import os
import pkg_resources as pkg


First, we need to initialize the logger here.

In [18]:
if 'SHARED_LOG_FILE' not in os.environ:
    os.environ['SHARED_LOG_FILE'] = f'straklip_{datetime.now().strftime("%Y-%m-%d_%H%M")}.log'

getLogger('straklip', setup=True, logfile=os.environ['SHARED_LOG_FILE'],debu=False,
          configfile=pkg.resource_filename('straklip', './config/logging.yaml'))

<Logger straklip (DEBUG)>

We import the 2 yalm files that holds all the options for the pipeline to run.

The data.yaml specify properties for the input catalogs that are needed to assemble te different dataframes for the pipeline.
The pipe.yaml instead, hold all the options that each step will need to perform its tasks.

Both yaml files need to be adjusted to reflect the specific project.

Since in the notebook we are explicitly calling each task we intend to run, we do not technically need the "flow" variable in the pipe.yaml, while it is necessary if we run the pipeline with the provide script "skpipe.yaml"

NOTE: the fits file needed to runt his tutorial ar not included at this moment, but they can be provided by private communication at any time.

In [19]:
pipe_cfg='/Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/pipe.yaml' #or where these files are
data_cfg='/Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/data.yaml'
# calls needed to configuration correctly the pipe_cfg and the data_cfg, that hare configurations need for the pipeline to work
pipe_cfg = config.configure_pipeline(pipe_cfg,pipe_cfg=pipe_cfg,data_cfg=data_cfg,dt_string=datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
data_cfg = config.configure_data(data_cfg,pipe_cfg)
config.make_paths(config=pipe_cfg)

2025-06-05 12:17:38 config                      :INFO     (configure_pipeline:72[pid=46866]) 
StraKLIP pipeline started at date and time: 05/06/2025 12:17:38
Pipe_cfg: /Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/pipe.yaml
Data_cfg: /Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/data.yaml

2025-06-05 12:17:38 config                      :INFO     (configure_data:153[pid=46866]) Validation of default labels and data successful!
2025-06-05 12:17:38 config                      :INFO     (make_paths:117[pid=46866]) "/Users/gstrampelli/StraKLIP/Tutorial/data/fits" exists, and will not be created.
2025-06-05 12:17:38 config                      :INFO     (make_paths:117[pid=46866]) "/Users/gstrampelli/StraKLIP/Tutorial/database" exists, and will not be created.
2025-06-05 12:17:38 config                      :INFO     (make_paths:117[pid=46866]) "/Users/gstrampelli/StraKLIP/Tutorial/out" exists, and will not be created.
2025-06-05 12:17:38 config                      :INFO     (mak

Once the pipe_cfg and data_cfg are ready, we can use them to finally configure the dataset that is a class that holds basic generic information that the pipeline can use to build its specific datasets. Since this is the first time we run the pipeline, we use skip_originals False to tell the pipeline to look for existing input photometry tables provided by the user instead of existing dataframes generated by the pipeline itself.

In [20]:
dataset = input_tables.Tables(data_cfg, pipe_cfg, skip_originals=False)
DF = config.configure_dataframe(dataset)

2025-06-05 12:17:38 config                      :WARNING  (configure_dataframe:226[pid=46866]) get_Av_dict currently only supports VEGAmag system. Please provide your own set of AVs if in a differest system as AVs : {ext: {mag_filter : value}} in the data.yaml under target
2025-06-05 12:17:38 ancillary                   :INFO     (get_Av_dict:425[pid=46866]) before dust, V =  0.0 mag(VEGA)
2025-06-05 12:17:38 ancillary                   :INFO     (get_Av_dict:426[pid=46866]) after dust, V = 1.0146 mag(VEGA)
2025-06-05 12:17:38 ancillary                   :INFO     (get_Av_dict:445[pid=46866]) Av = 1.0146 mag
2025-06-05 12:17:38 ancillary                   :INFO     (get_Av_dict:489[pid=46866]) AV=0 wfc3,uvis2,f814w 0.0 mag(VEGA)
2025-06-05 12:17:38 ancillary                   :INFO     (get_Av_dict:490[pid=46866]) AV=1 wfc3,uvis2,f814w 0.6094 mag
2025-06-05 12:17:38 ancillary                   :INFO     (get_Av_dict:489[pid=46866]) AV=0 wfc3,uvis2,f850lp 0.0 mag(VEGA)
2025-06-05 12:17:

Now we un the very first step of the pipeline: "buildhdf" where the pipeline will draw the needed information from the "original" input catalogs provided by the user, to assemble its specific dataframe and store them in the DF object. Running this step will also automatically save the generated dataframe on disk on the "out" paths specified, according to the dataframe extension (either a csv file or an h5) provided in the pipe.yaml

In [21]:
buildhdf.run({'DF': DF, 'dataset': dataset})

2025-06-05 12:17:38 steps.buildhdf              :INFO     (run:29[pid=46866]) Initializing new dataframes.
2025-06-05 12:17:38 utils_dataframe             :INFO     (mk_crossmatch_ids_df:268[pid=46866]) Creating the cross match ids dataframe
2025-06-05 12:17:38 steps.buildhdf              :INFO     (mk_targets_df:9[pid=46866]) Creating the targets dataframe
2025-06-05 12:17:38 utils_dataframe             :INFO     (mk_mvs_targets_df:354[pid=46866]) Creating the average targets dataframe
2025-06-05 12:17:38 utils_dataframe             :INFO     (mk_unq_targets_df:304[pid=46866]) Creating the multi-visit targets dataframe
2025-06-05 12:17:38 ancillary                   :INFO     (distances_cube:230[pid=46866]) Making the distance cube
2025-06-05 12:17:38 steps.buildhdf              :INFO     (make_candidates_dataframes:22[pid=46866]) Creating the candidates dataframe
2025-06-05 12:17:38 dataframe                   :INFO     (save_dataframes:109[pid=46866]) Saving the the following keys i

We can now look at the content of the DF object, and find different pandas dataframes in it:
  - crossmatch_ids_df: since the pipeline can wok with multiple visits surveys, this dataframe will store the which ids in the "unq" dataframes correspond to the ids in the "mvs" dataframes
  - mvs_targets_df: a type of "mvs" dataframe. "mvs" stand for multi-visits and store the basic information of your targets coming from multiple visits (so the same unqiue source could be stored multiple time if observed across multiple recurring visits)
  - unq_targets_df: a type of "unq" dataframe. "unq" stand for unique and store the  information of your targets gathered across multiple visits and averaged together. In  this kind of dataframes, there can be only one source for each id. If your survey consist in only one visit, then the "mvs" dataframes and the "unq" dataframes will be very similar, even though the typo of information stored may vary accorded to the pipeline needs (like is the case in this tutorial).
  - mvs_candidates_df: a type of "mvs" dataframe. It will store the properties of your candidates gatered from your mvs_targets_df when detected. Now empty.
  - unq_candidates_df: a type of "unq" dataframe. It will store the properties of your candidates gatered from your unq_targets_df when detected. Now empty.


In [22]:
DF.keys

['unq_targets_df',
 'crossmatch_ids_df',
 'mvs_targets_df',
 'mvs_candidates_df',
 'unq_candidates_df']

In [23]:
DF.mvs_targets_df

,mvs_ids,x_f814w,x_f850lp,y_f814w,y_f850lp,vis,ext,counts_f814w,counts_f850lp,ecounts_f814w,...,exptime_f814w,exptime_f850lp,cell_f814w,cell_f850lp,rota_f814w,rota_f850lp,pav3_f814w,pav3_f850lp,fits_f814w,fits_f850lp
0,0,766.682062,766.297865,870.519962,870.863230,13,1,NaN,NaN,NaN,...,712.0,712.0,NaN,NaN,169.438303,169.438303,124.436996,124.436996,iexn13010,iexn13020
1,1,769.100793,768.870891,866.146099,866.488792,01,1,NaN,NaN,NaN,...,712.0,712.0,NaN,NaN,142.364009,142.364009,97.360497,97.360497,iexn01010,iexn01020
2,2,762.415167,762.213142,869.798303,870.180384,02,1,NaN,NaN,NaN,...,716.0,716.0,NaN,NaN,143.685147,143.685147,98.681168,98.681168,iexn02010,iexn02020
3,3,765.630228,765.408206,867.979551,868.467341,03,1,NaN,NaN,NaN,...,712.0,712.0,NaN,NaN,146.062551,146.062551,101.059700,101.059700,iexn03010,iexn03020
4,4,767.251059,767.009600,867.473929,867.939866,04,1,NaN,NaN,NaN,...,712.0,712.0,NaN,NaN,144.993916,144.993916,99.990402,99.990402,iexn04010,iexn04020
5,5,766.601507,766.336710,868.267308,868.601284,05,1,NaN,NaN,NaN,...,716.0,716.0,NaN,NaN,145.422670,145.422670,100.418800,100.418800,iexn05010,iexn05020
6,6,767.081804,766.867054,866.901849,867.166727,06,1,NaN,NaN,NaN,...,712.0,712.0,NaN,NaN,145.295285,145.295285,100.292198,100.292198,iexn06010,iexn06020
7,7,765.825833,765.608929,868.625604,869.098238,07,1,NaN,NaN,NaN,...,712.0,712.0,NaN,NaN,145.278236,145.278236,100.274902,100.274902,iexn07010,iexn07020
8,8,765.740929,765.619928,868.295724,868.761725,08,1,NaN,NaN,NaN,...,712.0,712.0,NaN,NaN,145.102367,145.102367,100.098801,100.098801,iexn08010,iexn08020
9,9,762.743761,762.620949,866.727766,866.796443,09,1,NaN,NaN,NaN,...,712.0,712.0,NaN,NaN,144.710595,144.710595,99.707359,99.707359,iexn09010,iexn09020


In [24]:
DF.unq_targets_df

,unq_ids,ra,dec,m_f814w,m_f850lp,e_f814w,e_f850lp,type,FirstDist,SecondDist,ThirdDist,FirstID,SecondID,ThirdID,m_f814w_o,m_f850lp_o,e_f814w_o,e_f850lp_o
0,0,237.959171,-21.582686,NaN,NaN,NaN,NaN,1,4402.415542,7236.026079,10880.284451,13.0,7.0,23.0,22.277353,21.078353,0.021258,0.022984
1,1,251.650498,-23.227155,NaN,NaN,NaN,NaN,1,8788.229527,19936.394844,20713.866132,20.0,17.0,5.0,20.840983,19.749264,0.005901,0.006932
2,2,238.569423,-26.505131,NaN,NaN,NaN,NaN,1,5213.748442,7932.959139,9461.562793,26.0,8.0,14.0,21.448087,20.330281,0.010977,0.012019
3,3,241.686092,-20.561958,NaN,NaN,NaN,NaN,1,3968.240127,4083.676911,4368.932120,24.0,15.0,29.0,21.493397,20.318072,0.011936,0.012070
4,4,243.644804,-24.326008,NaN,NaN,NaN,NaN,1,1560.494131,1596.171284,4385.054415,27.0,28.0,12.0,21.553108,20.299904,0.011095,0.011329
5,5,247.042901,-26.673437,NaN,NaN,NaN,NaN,1,12025.901612,14646.516959,14769.965630,20.0,28.0,27.0,21.419805,20.159706,0.010367,0.010190
6,6,241.881676,-21.781889,NaN,NaN,NaN,NaN,1,3013.003477,3201.719746,3279.511440,9.0,24.0,21.0,22.182586,20.929665,0.022100,0.021179
7,7,238.932269,-23.341438,NaN,NaN,NaN,NaN,1,4120.677455,7236.026079,7593.796513,13.0,0.0,8.0,21.854202,20.710080,0.015996,0.016869
8,8,237.313809,-24.694252,NaN,NaN,NaN,NaN,1,3432.552745,7593.796513,7932.959139,22.0,7.0,2.0,22.007757,20.915487,0.017982,0.020411
9,9,242.327752,-22.490052,NaN,NaN,NaN,NaN,1,2326.651434,3013.003477,3130.865728,25.0,6.0,21.0,22.001171,20.883590,0.018724,0.020491


Now that the default dataframes are ready, we can run the next step in the pipeline: "mktiles".

With this step we generate a small tile around each target in our mvs_targets_df to define a search area for the pipeline.
This step will create a "mvs_tiles" and "median_tiles" folders in the out directory with inside a folder for each filter and all the corresponding tiles. The folder referring to the unq targets is called "median_tiles" because these tiles are generated taking the median of all the tiles for the same target in the "mvs_tiles" folder. This step will generate a fits cube for each source, containing a SCI image with the target at the center of the tile, an ERR and DQ cut out form the same coordinates, and if requested, also cosmic ray cleaned SCI images to use for the upcoming PSF subtraction.

As always, the dataframes are saved at the end of the step in the out directory.

In [25]:
mktiles.run({'DF': DF, 'dataset': dataset})

2025-06-05 12:17:39 steps.mktiles               :INFO     (make_mvs_tiles:316[pid=46866]) Working on the tiles
2025-06-05 12:17:39 steps.mktiles               :INFO     (mk_mvs_tiles:242[pid=46866]) Working on multi-visits tiles on filter f814w
2025-06-05 12:17:39 straklip.config             :INFO     (make_paths:117[pid=46866]) "/Users/gstrampelli/StraKLIP/Tutorial/out/mvs_tiles/f814w" exists, and will not be created.
2025-06-05 12:17:39 steps.mktiles               :INFO     (task_mvs_tiles:131[pid=46866]) Making mvs tile /Users/gstrampelli/StraKLIP/Tutorial/out/mvs_tiles/f814w/tile_ID52.fits
2025-06-05 12:17:39 steps.mktiles               :INFO     (make_median_tiles:420[pid=46866]) Working on median tiles on f814w
2025-06-05 12:17:39 straklip.config             :INFO     (make_paths:117[pid=46866]) "/Users/gstrampelli/StraKLIP/Tutorial/out/median_tiles/f814w" exists, and will not be created.
2025-06-05 12:17:39 steps.mktiles               :INFO     (task_median_tiles:356[pid=46866])

Now we can run the next step in the pipeline: "mkphotometry".

This step wil perform basic aperture photometry around each sources in the mvs_targets_df, providing vital information for the next stpes in the pipeline. A "targets_photometry_tiles" folder will be created in the "database" directory to store a quick visual summary of the photometry for each "mvs" target.

As always, the dataframes are saved at the end of the step in the out directory.

In [26]:
mkphotometry.run({'DF': DF, 'dataset': dataset})

2025-06-05 12:17:39 steps.mkphotometry          :INFO     (get_ee_df:25[pid=46866]) Fetching encircled energy dataframe for filters ['f814w', 'f850lp']
2025-06-05 12:17:39 steps.mkphotometry          :INFO     (make_mvs_photometry:75[pid=46866]) Make photometry for multi-visits targets on filter f814w
2025-06-05 12:17:39 ancillary                   :INFO     (parallelization_package:969[pid=46866]) Max allowable workers 8, # of elements 57 , # of chunk 19 approx # of elemtent per chunks 3 (chunksize)
2025-06-05 12:17:47 steps.mkphotometry          :INFO     (make_median_photometry:147[pid=46866]) Make photometry for average targets on filter f814w
2025-06-05 12:17:47 ancillary                   :INFO     (parallelization_package:969[pid=46866]) Max allowable workers 8, # of elements 57 , # of chunk 19 approx # of elemtent per chunks 3 (chunksize)
2025-06-05 12:17:50 steps.mkphotometry          :INFO     (make_mvs_photometry:75[pid=46866]) Make photometry for multi-visits targets on fil

Now we can check again our dataframes, and we can see that the columns related to the photometry have been populated.

In [27]:
DF.mvs_targets_df

,mvs_ids,x_f814w,x_f850lp,y_f814w,y_f850lp,vis,ext,counts_f814w,counts_f850lp,ecounts_f814w,...,exptime_f814w,exptime_f850lp,cell_f814w,cell_f850lp,rota_f814w,rota_f850lp,pav3_f814w,pav3_f850lp,fits_f814w,fits_f850lp
0,0,766.682062,766.297865,870.519962,870.863230,13,1,10072.242444,8147.884608,104.222556,...,712.0,712.0,NaN,NaN,169.438303,169.438303,124.436996,124.436996,iexn13010,iexn13020
1,1,769.100793,768.870891,866.146099,866.488792,01,1,37148.970247,29523.416694,200.603377,...,712.0,712.0,NaN,NaN,142.364009,142.364009,97.360497,97.360497,iexn01010,iexn01020
2,2,762.415167,762.213142,869.798303,870.180384,02,1,20888.773322,16756.185268,150.306225,...,716.0,716.0,NaN,NaN,143.685147,143.685147,98.681168,98.681168,iexn02010,iexn02020
3,3,765.630228,765.408206,867.979551,868.467341,03,1,19851.347253,17782.298850,146.991839,...,712.0,712.0,NaN,NaN,146.062551,146.062551,101.059700,101.059700,iexn03010,iexn03020
4,4,767.251059,767.009600,867.473929,867.939866,04,1,18535.997643,17143.439224,141.164681,...,712.0,712.0,NaN,NaN,144.993916,144.993916,99.990402,99.990402,iexn04010,iexn04020
5,5,766.601507,766.336710,868.267308,868.601284,05,1,21899.906087,19840.838076,152.199777,...,716.0,716.0,NaN,NaN,145.422670,145.422670,100.418800,100.418800,iexn05010,iexn05020
6,6,767.081804,766.867054,866.901849,867.166727,06,1,10821.808262,9736.723197,111.510075,...,712.0,712.0,NaN,NaN,145.295285,145.295285,100.292198,100.292198,iexn06010,iexn06020
7,7,765.825833,765.608929,868.625604,869.098238,07,1,14345.541776,12125.508627,123.726601,...,712.0,712.0,NaN,NaN,145.278236,145.278236,100.274902,100.274902,iexn07010,iexn07020
8,8,765.740929,765.619928,868.295724,868.761725,08,1,12134.206337,9881.013676,116.470224,...,712.0,712.0,NaN,NaN,145.102367,145.102367,100.098801,100.098801,iexn08010,iexn08020
9,9,762.743761,762.620949,866.727766,866.796443,09,1,12163.774100,9674.484288,118.946485,...,712.0,712.0,NaN,NaN,144.710595,144.710595,99.707359,99.707359,iexn09010,iexn09020


Last step needed for the data reduction is the: "fow2cells".

To avid distortion of the PSF across the fild of view, we generate a grid and group together close by stars to use as references for the upcoming PSF subtraction. For this tutorial, since all our sources are at the center of the FOW, we will need a very basic grid for this, but ideally we could generate a finer grid, as long as there are enough good reference stars in each cell to be able to run the PSF subtraction.
As always, the dataframes are saved at the end of the step in the out directory.


In [28]:
fow2cells.run({'DF': DF, 'dataset': dataset})

2025-06-05 12:18:00 steps.fow2cells             :INFO     (break_FOW_in_cells:18[pid=46866]) Braking f814w FOW in 1 cells.
2025-06-05 12:18:01 utils_plot                  :INFO     (fow_stamp:312[pid=46866]) Saving cell_f814w.png in /Users/gstrampelli/StraKLIP/Tutorial/database/
2025-06-05 12:18:01 steps.fow2cells             :INFO     (break_FOW_in_cells:18[pid=46866]) Braking f850lp FOW in 1 cells.
2025-06-05 12:18:01 utils_plot                  :INFO     (fow_stamp:312[pid=46866]) Saving cell_f850lp.png in /Users/gstrampelli/StraKLIP/Tutorial/database/
2025-06-05 12:18:01 steps.fow2cells             :INFO     (run:80[pid=46866]) Updating type for unique detections.
2025-06-05 12:18:01 dataframe                   :INFO     (save_dataframes:109[pid=46866]) Saving the the following keys in ['unq_targets_df', 'crossmatch_ids_df', 'mvs_targets_df', 'mvs_candidates_df', 'unq_candidates_df'] to .csv files in /Users/gstrampelli/StraKLIP/Tutorial/out


In [29]:
DF.mvs_targets_df


,mvs_ids,x_f814w,x_f850lp,y_f814w,y_f850lp,vis,ext,counts_f814w,counts_f850lp,ecounts_f814w,...,exptime_f814w,exptime_f850lp,cell_f814w,cell_f850lp,rota_f814w,rota_f850lp,pav3_f814w,pav3_f850lp,fits_f814w,fits_f850lp
0,0,766.682062,766.297865,870.519962,870.863230,13,1,10072.242444,8147.884608,104.222556,...,712.0,712.0,0.0,0.0,169.438303,169.438303,124.436996,124.436996,iexn13010,iexn13020
1,1,769.100793,768.870891,866.146099,866.488792,01,1,37148.970247,29523.416694,200.603377,...,712.0,712.0,0.0,0.0,142.364009,142.364009,97.360497,97.360497,iexn01010,iexn01020
2,2,762.415167,762.213142,869.798303,870.180384,02,1,20888.773322,16756.185268,150.306225,...,716.0,716.0,0.0,0.0,143.685147,143.685147,98.681168,98.681168,iexn02010,iexn02020
3,3,765.630228,765.408206,867.979551,868.467341,03,1,19851.347253,17782.298850,146.991839,...,712.0,712.0,0.0,0.0,146.062551,146.062551,101.059700,101.059700,iexn03010,iexn03020
4,4,767.251059,767.009600,867.473929,867.939866,04,1,18535.997643,17143.439224,141.164681,...,712.0,712.0,0.0,0.0,144.993916,144.993916,99.990402,99.990402,iexn04010,iexn04020
5,5,766.601507,766.336710,868.267308,868.601284,05,1,21899.906087,19840.838076,152.199777,...,716.0,716.0,0.0,0.0,145.422670,145.422670,100.418800,100.418800,iexn05010,iexn05020
6,6,767.081804,766.867054,866.901849,867.166727,06,1,10821.808262,9736.723197,111.510075,...,712.0,712.0,0.0,0.0,145.295285,145.295285,100.292198,100.292198,iexn06010,iexn06020
7,7,765.825833,765.608929,868.625604,869.098238,07,1,14345.541776,12125.508627,123.726601,...,712.0,712.0,0.0,0.0,145.278236,145.278236,100.274902,100.274902,iexn07010,iexn07020
8,8,765.740929,765.619928,868.295724,868.761725,08,1,12134.206337,9881.013676,116.470224,...,712.0,712.0,0.0,0.0,145.102367,145.102367,100.098801,100.098801,iexn08010,iexn08020
9,9,762.743761,762.620949,866.727766,866.796443,09,1,12163.774100,9674.484288,118.946485,...,712.0,712.0,0.0,0.0,144.710595,144.710595,99.707359,99.707359,iexn09010,iexn09020


With this step the "cell_{filter}" columns of the "mvs_targets_df" is populated, and the data reduction is completed and the primary dataframes are assembled.

We are now ready to move on to the PSF subtraction section of the pipeline.